# Analysis of bolometric light curves of type Ia SNe
This notebook allows a simple yet accurate analysis of bolometric light curves of type Ia supernovae. 
Feel free to copy or modify. A folder with an updated version of this notebook and a few light-curve input files from the literature can be obtained [here](https://www.dropbox.com/sh/hlidmb06oqrtu8c/AAATE3nsbJi6ziJY5abY5oB2a?dl=0).

### First load relevant libraries and functions
In order to use this notebook, you need to have python istalled along with some standard libraries. The anaconda package is sufficient. Run the following cell before using the notebook (click in the cell and then press shift+enter).


In [ ]:
%pylab
from pandas import read_csv
from scipy.integrate import cumtrapz
from ipywidgets import interact

### Define Energy release in Ni56->Co56->Fe56 decay, gamma+pos:
First, run the following cell in order to define the total release of energy from the 56Ni decay chain and approximate deposition from gammas and positrons:

In [ ]:
def QNidec_ga(td,MNi):  # total power released to gamma-rays (including the e+e- anihilation line assuming total positron anihilation)
    return MNi*(6.45*exp(-td/8.76)+1.38*exp(-td/111.4))*1e43  

def QNidec_pos(td,MNi):  # total power released to positron kinetic energy
    return MNi*4.64*(-exp(-td/8.76)+exp(-td/111.4))*1e41  

def QNidep(td,MNi,t0): # total deposition rate by gamma-rays and positrons taking into account gamma-ray escape, assuming full deposition of positron kinetic energy
    fdep_ga=(1-exp(-t0**2/td**2))
    return QNidec_ga(td,MNi)*fdep_ga+QNidec_pos(td,MNi)

For QNidep (deposition), it is assumed that positron kinetic energy is fully deposited. At early times the deposition fraction of gamma-rays is unity. At late times, when the ejecta is optically thin, the deposition fraction scales as 1/t^2 and is parametrized as t0^2/t^2 with t0 a parameter set by the ejecta (see e.g. [Jeferry1999](http://arxiv.org/abs/astro-ph/9907015) and [Clocchiatti&Wheeler1997](http://adsabs.harvard.edu/abs/1997ApJ...491..375C)).  The interpolation, fdep=1-exp(-t0^2/t^2), is used which is an excellent approximation for most ejeca models. 

### Create light curve analysis gui functions.
Next run the following cell to set some initial values (you will fit those later) and create the gui function.

In [ ]:
## initial values that can be modified later.
MNi_init=0.5      #initial value for Nickel 56 Mass [Msun]

texp_init=0.0  #initial value of time of explosion 
                #relative to first data point [days]

t0_init=40    #initial value for Jeffereys gamma ray t0 [days]  
                #(relevant only when gamma escape is included)
    
FS=15           #font size of title and labels

    ##gui function including gamma-ray escape (crucial for type Ia):
def LtIntGamma(td,log10L,MNimax,dMNi,t0max,dt0,texpmin,texpmax):
    # preparing figure
    #######################   
    fig=figure(1); clf()
    ax1=fig.add_subplot(311) 
    dataL,=ax1.plot(0,0,'b.')
    line1,=ax1.plot(0,0,'r')
    line1.set_zorder(100)
    ax1.set_ylabel('log10(L[ergs/s])',fontsize=FS)
    ax2=fig.add_subplot(312)
    dataInt,=ax2.plot(0,0,'b.')
    line2,=ax2.plot(0,0,'r')
    line2.set_zorder(100)
    ax2.set_ylabel('Int Ltdt [ergs d]',fontsize=FS)
    ax3=fig.add_subplot(313)
    dataR,=ax3.plot(0,1,'b.')
    line3,=ax3.plot(0,0,'r')
    line3.set_zorder(100)
    ax3.set_xlabel('t[d]',fontsize=FS);
    ax3.set_ylabel('t^2.5 L/Int Ltdt [d^0.5]',fontsize=FS)
    ####################### setting the axis limits
    t=td-texp_init                    # just for setting axes limits
    IntLtdt=cumtrapz(L*t,t,initial=0) # just for setting axes limits
    ax1.set_xlim(0,tmax); 
    ax1.set_ylim(min(log10L[(t<tmax)*(t>5)])-0.5,max(log10L)+0.5)
    ax2.set_xlim(0,tmax)
    ax2.set_ylim(0,max(IntLtdt[t<tmax]*86400)*1.5)
    ax3.set_xlim(0,tmax)
    ax3.set_ylim(4,10)
    #######################
    
    #here the fit and actual plot happens:
    def tryMNi(MNi=MNi_init,t0=t0_init,texp=texp_init):
        t=td-texp                           # calculating the time since explosion
        dataL.set_data(t,log10L)            # plotting the data in the top subplot
        IntLtdt=cumtrapz(L*t,t,initial=0)   # calculating the cumulative (time weighted) emitted light curve
        dataInt.set_data(t,IntLtdt*86400)   # plotting the data in the middle subplot
        R=t**2.5*L/IntLtdt                  # calculating from the data the ratio R.5=t^2.5*L/integral
        dataR.set_data(t,R)                 # plotting R.5 from the data in the bottom panel 
        tp=arange(0,tmax)                   # time variable used for analytic estimates
        Qd=QNidep(tp,MNi,t0)                # calculating the deposition function taking into acount gamma escape
        IntQtdt=cumtrapz(Qd*tp,tp,initial=0)# calculating the cumulative (time-weighted) deposition
        Rd=tp**2.5*Qd/IntQtdt               # calculating R.5 from the deposition function
        line1.set_data(tp,log10(Qd))        # plotting the deposition function in the top subplot
        line2.set_data(tp,IntQtdt*86400)    # plotting the cumulative (time-weighted) deposition in the middle subplot
        line3.set_data(tp,tp**2.5*Qd/IntQtdt) # plotting R.5 for the deposition function 
        ax1.set_title('Ni56='+str(MNi)+' Msun '+' t0_gamma='+str(t0)+' days',fontsize=FS)
        global MNi_init,t0_init,texp_init; MNi_init=MNi; t0_init=t0; texp_init=texp;
    interact(tryMNi,MNi=(0,MNimax,dMNi),t0=(15,t0max,dt0),texp=(texpmin,texpmax,0.1))
    return ax1,ax2,ax3

### Analyze type Ia bolometric light curves.

Run the following cell to analyze a specific supernova by manually fitting the gamma-ray escape time t0 and the total Ni56 yield MNi using the slidebars (you can use the arrows on the keyboard for small adjustments). You can also zoom in any of the panels. The sensitivity to the (usually not precisely known) time of explosion texp can be studied by changing it.  Note that the ratio in the bottom panel does not depend on the amount of Ni56 and depends only on t0 allowing t0 to be obtained by fitting. Then the top panels can be used to find the amount of Ni56.
If you create a new light curve file, each row should have two numbers separated by a comma:
t - time in days, it is assumed that t=0 is at or after explosion.
log10L=log10(Lbol), where Lbol is the bolometric luminosity in erg/s. The file should have *two* header text lines (typically the first mentions the source and the second is the titles to the two columns e.g. t[days],log10(L).

In [ ]:
###### Analyze a lightcurve
# available type Ia files: sn2005cf_Wang09.txt, sn2003du_Stanishev07.txt, 
# sn2011fe_Mazzali15.txt, sn2007on_Phillips12.txt, sn2005ke_Phillips12.txt 

file='sn2005cf_Wang09.txt'   


SN=read_csv(file,names=['t','log10L'],header=1) 
td=array(SN['t'])
log10L=array(SN['log10L'])

MNimax=1;      # maximal value allowed for 56Ni [Msun]
dMNi=0.001;     # resolution of 56Ni mass [Msun]
tmax=100;       # limit of presented time [days]
t0max=100;     # maximal gamma escape time t0 [days]
dt0=0.1;      # resolution of t0 [days]
texpmin=-20;    # minimal time of explosion in days
texpmax=20;    # maximal time of explosion in days

N=len(td); L=10**log10L;
ax1,ax2,ax3=LtIntGamma(td,log10L,MNimax,dMNi,t0max,dt0,texpmin,texpmax)